# Pydeck Earth Engine Introduction

This is an introduction to using [Pydeck](https://pydeck.gl) and [Deck.gl](https://deck.gl) with [Google Earth Engine](https://earthengine.google.com/) in Jupyter Notebooks.

If you wish to run this locally, you'll need to install some dependencies. Installing into a new Conda environment is recommended. To create and enter the environment, run:
```
conda create -n pydeck-ee -c conda-forge python jupyter notebook pydeck earthengine-api requests -y
source activate pydeck-ee
jupyter nbextension install --sys-prefix --symlink --overwrite --py pydeck
jupyter nbextension enable --sys-prefix --py pydeck
```
then open Jupyter Notebook with `jupyter notebook`.

Now in a Python Jupyter Notebook, let's first import required packages:

In [1]:
from pydeck_earthengine_layers import EarthEngineLayer
import pydeck as pdk
import requests
import ee

## Authentication

Using Earth Engine requires authentication. If you don't have a Google account approved for use with Earth Engine, you'll need to request access. For more information and to sign up, go to https://signup.earthengine.google.com/.

If you haven't used Earth Engine in Python before, you'll need to run the following authentication command. If you've previously authenticated in Python or the command line, you can skip the next line.

Note that this creates a prompt which waits for user input. If you don't see a prompt, you may need to authenticate on the command line with `earthengine authenticate` and then return here, skipping the Python authentication.

In [ ]:
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

## Create Map

Next it's time to create a map. Here we create an `ee.Image` object

In [ ]:
# Initialize objects
ee_layers = []
view_state = pdk.ViewState(latitude=37.7749295, longitude=-122.4194155, zoom=10, bearing=0, pitch=45)

In [4]:
# %%
# Add Earth Engine dataset
def intersect(state):
  nPowerPlants = ee.List(state.get('power_plants')).size()
  # Return the state feature with a new property: power plant count.
  return state.set('n_power_plants', nPowerPlants)

# Load the primary 'collection': US state boundaries.
states = ee.FeatureCollection('TIGER/2018/States')

# Load the secondary 'collection': power plants.
powerPlants = ee.FeatureCollection('WRI/GPPD/power_plants')

# Define a spatial filter as geometries that intersect.
spatialFilter = ee.Filter.intersects(**{
  'leftField': '.geo',
  'rightField': '.geo',
  'maxError': 10
})

# Define a save all join.
saveAllJoin = ee.Join.saveAll(**{
  'matchesKey': 'power_plants',
})

# Apply the join.
intersectJoined = saveAllJoin.apply(states, powerPlants, spatialFilter)

# Add power plant count per state as a property.
intersectJoined = intersectJoined.map(intersect)
# intersectJoined = intersectJoined.map(function(state) {
#   # Get "power_plant" intersection list, count how many intersected this state.
#   nPowerPlants = ee.List(state.get('power_plants')).size()
#   # Return the state feature with a new property: power plant count.
#   return state.set('n_power_plants', nPowerPlants)
# })

print(intersectJoined.getInfo())

# # Make a bar chart for the number of power plants per state.
# chart = ui.Chart.feature.byFeature(intersectJoined, 'NAME', 'n_power_plants') \
#   .setChartType('ColumnChart') \
#   .setSeriesNames({n_power_plants: 'Power plants'}) \
#   .setOptions({
#     title: 'Power plants per state',
#     hAxis: {title: 'State'},
#     vAxis: {title: 'Frequency'}})

# # Print the chart to the console.
# print(chart)




Then just pass these layers to a `pydeck.Deck` instance, and call `.show()` to create a map:

In [8]:
r = pdk.Deck(layers=ee_layers, initial_view_state=view_state)
r.show()

DeckGLWidget(custom_libraries=[{'libraryName': 'EarthEngineLayerLibrary', 'resourceUri': 'https://cdn.jsdelivr…